In [202]:
import email
import hashlib
import imaplib
import os
from collections import defaultdict, Counter
import platform
from gmailsecret import password, userName

In [204]:
mail = imaplib.IMAP4_SSL('imap.gmail.com')
typ, accountDetails = mail.login(userName, password)

In [205]:
print(typ)
print(accountDetails)

OK
[b'jason.becker86@gmail.com authenticated (Success)']


In [206]:
mail.list()

('OK',
 [b'(\\HasNoChildren) "/" "Bath and Body Works"',
  b'(\\HasNoChildren) "/" "Berkeley"',
  b'(\\HasNoChildren) "/" "Downloaded"',
  b'(\\HasNoChildren) "/" "Forwards"',
  b'(\\HasNoChildren) "/" "Future Job"',
  b'(\\HasNoChildren) "/" "Great Dane"',
  b'(\\HasNoChildren) "/" "Groupon"',
  b'(\\HasNoChildren) "/" "INBOX"',
  b'(\\HasNoChildren) "/" "Lydia"',
  b'(\\HasNoChildren) "/" "Montana"',
  b'(\\HasNoChildren) "/" "Notes"',
  b'(\\HasNoChildren) "/" "Ordering Stuff"',
  b'(\\HasNoChildren) "/" "Summer Science Camp 08"',
  b'(\\HasNoChildren) "/" "Teaching ideas"',
  b'(\\HasNoChildren) "/" "VideoGame Making"',
  b'(\\HasNoChildren) "/" "West Millbrook"',
  b'(\\HasChildren \\Noselect) "/" "[Gmail]"',
  b'(\\All \\HasNoChildren) "/" "[Gmail]/All Mail"',
  b'(\\Drafts \\HasNoChildren) "/" "[Gmail]/Drafts"',
  b'(\\HasNoChildren \\Important) "/" "[Gmail]/Important"',
  b'(\\HasNoChildren \\Sent) "/" "[Gmail]/Sent Mail"',
  b'(\\HasNoChildren \\Junk) "/" "[Gmail]/Spam"',
  b'

In [207]:
mail.select('inbox')
typ, data = mail.search(None, '(X-GM-RAW "has:attachment")')
print(data[0])

b'7 8 11 12 13 14 16 17 19 20 21 22 23 24 25 30 37 38 39 45 49 50 57 59 60 61 69 71 73 74 79 154 155 158 188 215 248 249 287 324 348 407 424 425 434 435 450 451 484 487 525 551 555 558 575 578 588 595 612 642 647 661 683 741 747 748 751 758 765 834 863 867 880 881 890 914 921 928 932 989 991 1001 1039 1047 1052 1053 1062 1082 1084 1087 1091 1096 1121 1122 1125 1131 1132 1137 1142 1146 1180 1198 1203 1218 1235 1260 1289 1311 1312 1322 1331 1333 1360 1368 1386 1394 1395 1399 1413 1418 1440 1441 1460 1476 1493 1495 1501 1509 1512 1530 1531 1630 1674 1675 1686 1693 1751 1752 1753 1754 1758 1760 1770 1773 1782 1806 1827 1838 1849 1858 1869 1877 1878 1900 1901 1903 1904 1907 1908 1910 1911 1936 1952 1953 1959 1960 1961 1962 1963 1964 1965 1966 1967 1994 2100 2101 2102 2103 2104 2105 2108 2109 2110 2111 2112 2113 2125 2140 2149 2150 2151 2170 2171 2172 2173 2175 2176 2177 2179 2185 2201 2236 2237 2249 2257 2275 2280 2281 2282 2283 2284 2285 2286 2287 2288 2289 2313 2318 2319 2326 2327 2328 23

In [230]:
fileNameCounter = Counter()
for msgID in data[0].split()[-1:-2:-1]:
    typ, response = mail.fetch(msgID, '(X-GM-LABELS)')
    if any(['Downloaded' in str(s) for s in response]):
        print(response)
        typ, messageParts = mail.fetch(msgID, '(RFC822)')
        emailBody = messageParts[0][1]
        print(type(emailBody))
        finalEmail = email.message_from_string(str(emailBody, 'UTF-8'))
        if finalEmail.get_content_maintype() != 'multipart':
            print('Not multipart.  Skipping')
            continue
        for part in finalEmail.walk():
            #print(part)
            print("The next Part:")
            #print(part.as_string())
            if part.get_content_maintype() == 'multipart':
                print('Part is Multipart: Skipping')
                continue
            elif part.get('Content-Disposition') is None:
                print('Part has no Content-Disposition')
                continue
            fileName = part.get_filename()
            print(fileName)
            if fileName is not None:
                fileName = ''.join(fileName.splitlines())
            if fileName:
                print('Processing: {file}'.format(file=fileName))
                payload = part.get_payload(decode=True)
                if payload:
                    fileNameCounter[fileName] += 1
                    fileStr, fileExstension = os.path.splitext(fileName)
                    if fileNameCounter[fileName] > 1:
                        new_fileName = '{file}({suffix}){ext}'.format(suffix=fileNameCounter[fileName],
                                                                     file=fileStr, ext=fileExtension)
                        print('\tRenaming and storing: {file} to {new_file}'.format(file=fileName,
                                                                                   new_file=new_fileName))
                    else:
                        new_fileName = fileName
                        print('\tStoring: {file}'.format(file=fileName))
                    file_path = os.path.join('attachments', new_fileName)
                    if os.path.exists(file_path):
                        print('\tExists in destination: {file}'.format(file=new_fileName))
                        continue
                    try:
                        with open(file_path, 'wb') as fp:
                            fp.write(payload)
                    except:
                        print('Could not store')
    #for part in message:
    #    fileName = message.get_filename()
    #    print(fileName)
    #fileName = messageParts[0][1].get_filename()
    
    #print(fileName)
    #emailBody = messageParts[0][1]
    #print(emailBody)
    #message = email.message_from_string(str(emailBody))
    #print(message[:45])

[b'18684 (X-GM-LABELS ("\\\\Important" "\\\\Sent" Downloaded))']
<class 'bytes'>
The next Part:
Part is Multipart: Skipping
The next Part:
Part is Multipart: Skipping
The next Part:
Part has no Content-Disposition
The next Part:
Part has no Content-Disposition
The next Part:
__test2.xlsx
Processing: __test2.xlsx
	Storing: __test2.xlsx


In [ ]:
typ, messageParts = mail.fetch('18677', '(RFC822)')
print(messageParts)

In [ ]:
emailBody = messageParts[0][1]

In [ ]:
message = email.message_from_string(str(messageParts))

In [ ]:
for part in message.walk():
    #print(part)
    fileName = part.get_filename()
    if fileName is not None:
        print(fileName)
    else:
        print('no filename :(')

In [ ]:
print(message)